<a href="https://colab.research.google.com/github/MosheWasserb/PyTorchNotbooks/blob/master/STS2_BERT_FT_3_convert_to_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 573kB 2.6MB/s 
     |████████████████████████████████| 3.7MB 50.4MB/s 
     |████████████████████████████████| 1.0MB 34.9MB/s 
     |████████████████████████████████| 890kB 46.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=aa735e989ebdd5d59134a445c71fb1de04701abc96b4b09751a58222d82c186c
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn import preprocessing
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.preprocessing import LabelEncoder
from IPython.display import clear_output
from torch.nn.utils import clip_grad_norm_
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDClassifier

### Helper functions
import pickle

def convert_to_pickle(item, directory):
    pickle.dump(item, open(directory,"wb"))


def load_from_pickle(directory):
    return pickle.load(open(directory,"rb"))

Importing the dataset

In [5]:
# Import Training, Validation and Test csvs
# STS2
#data = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
# 6 Emotions
#data = pd.read_csv(r"/gdrive/My Drive/Colab Notebooks/DAIR/EmotionDataSet/multiclass6ed.csv")
# IMDB data (from Kaggle)
#data = pd.read_csv(r"/gdrive/My Drive/DataSet/IMDB/train2.csv")
# STS2 from GLUE 
data = pd.read_csv("/gdrive/My Drive/DataSet/GLUE/STS2/SST-2/train.tsv", delimiter='\t', header=None)

text_col=data.columns.values[0] 
category_col=data.columns.values[1]

data.head

<bound method NDFrame.head of                                                        0      1
0                                               sentence  label
1           hide new secretions from the parental units       0
2                   contains no wit , only labored gags       0
3      that loves its characters and communicates som...      1
4      remains utterly satisfied to remain the same t...      0
...                                                  ...    ...
67345                               a delightful comedy       1
67346                   anguish , anger and frustration       0
67347  at achieving the modest , crowd-pleasing goals...      1
67348                                  a patient viewer       1
67349  this new jangle of noise , mayhem and stupidit...      0

[67350 rows x 2 columns]>

Init and set data

In [0]:
TEST_SIZE=0.2
RANDOM_STATE=1
SAMPLE_SIZE = 8000
UNLABEL_SIZE = 5000
AUG_SIZE = 4000

max_bert_len = 70

BATCH_SIZE=64
EPOCHS =3
HIDDEN_SIZE=768

catagories=list(set(data[category_col].unique()))
OUTPUT_DIM= 2

In [0]:
#aug_data_sample=aug_data.sample(n=AUG_SIZE,random_state=RANDOM_STATE)
#aug_texts=aug_data_sample[0]

In [0]:
batch_1 = data.sample(n=SAMPLE_SIZE, random_state=RANDOM_STATE);
residue_batch = data.drop(batch_1.index)
unlabel_texts_batch = residue_batch.sample(n=UNLABEL_SIZE, random_state=RANDOM_STATE);
unlabel_texts=unlabel_texts_batch[text_col]

In [0]:
train_texts, test_texts, train_labels, test_labels = train_test_split(batch_1[text_col],batch_1[category_col], test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [85]:
list(set(test_labels.unique()))

['1', '0']

In [31]:
len(train_texts), len(test_texts), len(train_labels), len(test_labels)

(6400, 1600, 6400, 1600)

1. Ngram baseline model

In [32]:
#CounterVecorizer(ngram_range=(1,3), min_df=0.2, max_df=0.7, max_features=10000, stop_words="english")
ngramCount_baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)
#ngramCount_baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), LogisticRegression()).fit(train_texts, train_labels)
#ngramCount_baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,2),stop_words="english",max_features=30000, max_df=0.75), LogisticRegression()).fit(train_texts, train_labels)
ngramCount_baseline_predicted = ngramCount_baseline_model.predict(test_texts)
print(classification_report(test_labels, ngramCount_baseline_predicted))
print(confusion_matrix(test_labels, ngramCount_baseline_predicted))

              precision    recall  f1-score   support

           0       0.84      0.65      0.73       705
           1       0.77      0.90      0.83       895

    accuracy                           0.79      1600
   macro avg       0.80      0.78      0.78      1600
weighted avg       0.80      0.79      0.79      1600

[[458 247]
 [ 88 807]]


In [14]:
ngramCount_baseline_model.score(test_texts,test_labels)

0.791

1.1 NGRAM SGDClassifier 

In [15]:
ngramCount_baseline_model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), SGDClassifier(loss = 'log')).fit(train_texts, train_labels)
ngramCount_baseline_predicted = ngramCount_baseline_model.predict(test_texts)
print(classification_report(test_labels, ngramCount_baseline_predicted))
print(confusion_matrix(test_labels, ngramCount_baseline_predicted))

              precision    recall  f1-score   support

           0       0.77      0.66      0.71       175
           1       0.76      0.85      0.80       225

    accuracy                           0.77       400
   macro avg       0.77      0.75      0.76       400
weighted avg       0.77      0.77      0.76       400

[[115  60]
 [ 34 191]]


2. Glove mean Logistic

In [4]:
### read data from your Google Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [56]:
# define dict to hold a word and its vector
glove = {}
# read the word embeddings file ~820MB
f = open('/gdrive/My Drive/Colab Notebooks/DAIR/EmotionDataSet/glove.6B.100d.txt', encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove[word] = coefs
f.close()
# check the length
len(glove) # 400000

400000

In [0]:
def MeanSentenceGlove(texts):
# create vector for each sentences
# list to hold vector 
  sentence_vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      if len(i) != 0:
          v = sum([glove.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
          #print(len(i))
      else:
          v = np.zeros((100,))
      sentence_vectors.append(v)
  #print('Total vectors created:',len(sentence_vectors))
  return(sentence_vectors)

In [0]:
train_glove = MeanSentenceGlove(train_texts)
test_glove = MeanSentenceGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

    negative       0.72      0.80      0.76       209
    positive       0.75      0.66      0.71       191

    accuracy                           0.73       400
   macro avg       0.74      0.73      0.73       400
weighted avg       0.74      0.73      0.73       400



In [0]:
temp = MeanSentenceGlove(train_texts)

Total vectors created: 64


In [0]:
temp[0]

2.1 Glove - NLTK tokenize

In [54]:
 #We'll use Average Glove here 
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def AvgGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      #vectors.append(np.average(glove.get(word_tokenize(i)), axis =0))
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0))
  return np.array(vectors)

In [0]:
train_glove = AvgGlove(train_texts)
test_glove = AvgGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

    negative       0.74      0.81      0.78       209
    positive       0.77      0.70      0.73       191

    accuracy                           0.76       400
   macro avg       0.76      0.75      0.75       400
weighted avg       0.76      0.76      0.75       400



2.2 Glove Weighted average tfidf

In [0]:
tfidf = TfidfVectorizer()
tfidf.fit(train_texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
idf_dict = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [0]:
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.

def tfidfGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      weights = [idf_dict.get(word, 1) for word in word_tokenize(i)]
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0, weights = weights))
  return np.array(vectors)

In [0]:
train_glove = tfidfGlove(train_texts)
test_glove = tfidfGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

              precision    recall  f1-score   support

    negative       0.77      0.77      0.77       209
    positive       0.74      0.74      0.74       191

    accuracy                           0.76       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.76      0.76      0.76       400



2.3 Glove Weighted average Count

In [0]:
count = CountVectorizer()
count.fit(train_texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
# Now lets create a dict so that for every word in the corpus we have a corresponding IDF value
count_dict = count.vocabulary_

In [0]:
# Same as Avg Glove except instead of doing a regular average, we'll use the IDF values as weights.

def countGlove(texts):
# create vector for each sentences
# list to hold vector 
  vectors = []
# create vector for each clean normalized sentence
  for i in texts:
      weights = [count_dict.get(word, 1) for word in word_tokenize(i)]
      vectors.append(np.average([glove.get(w, np.zeros((100,))) for w in word_tokenize(i)], axis = 0, weights = weights))
  return np.array(vectors)

In [1]:
train_glove = countGlove(train_texts)
test_glove = countGlove(test_texts)
Glove_baseline_model=LogisticRegression()
Glove_baseline_model.fit(train_glove, train_labels)
Glove_baseline_predicted = Glove_baseline_model.predict(test_glove)
print(classification_report(test_labels, Glove_baseline_predicted))

NameError: ignored

3. FastText

In [0]:
from gensim.models import FastText

In [0]:
from gensim.utils import tokenize

In [0]:
list(tokenize('I like Moshe food', lowercase=True))

['i', 'like', 'moshe', 'food']

In [0]:
sentences_tokens=[[token for token in tokenize(sentence)] for sentence in train_texts]

In [0]:
model_fasttext_sts2 = FastText(sentences_tokens, size=100, window=5, min_count=0, workers=4,sg=0)

In [0]:
model_fasttext_sts2.most_similar('eat')


4. BERT Fine-tune

Load BERT pre-trained model

In [33]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want Roberta instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.RobertaModel, ppb.RobertaTokenizer, 'roberta-base')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
BertModel = model_class.from_pretrained(pretrained_weights)

Tokenization

In [0]:
import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)
tokenized = batch_1[text_col].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Limit tokenized to 512 max_bert_len

In [0]:
#padded = np.array([if (len(i)>0) i=3, for i in tokenized.values])
tokenized_limted = []
for i in tokenized.values:
  if len(i)>max_bert_len:
    tokenized_limted.append(i[:max_bert_len])
  else:
    tokenized_limted.append(i)
     


Padding 

In [0]:
padded = np.array([i + [0]*(max_bert_len-len(i)) for i in tokenized_limted])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [37]:
np.array(padded).shape

(8000, 70)

Masking

In [38]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(8000, 70)

Convert to tensors

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [33]:
attention_mask

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])

**And now Deep Learning**

Biniarizatiom

In [0]:
mlb = preprocessing.MultiLabelBinarizer()
data_labels =  [set(catos) & set(catagories) for catos in batch_1[[category_col]].values]
bin_catagories = mlb.fit_transform(data_labels)
target_tensor_bin = torch.tensor(bin_catagories.tolist())

target_tensor=torch.max(target_tensor_bin,1)[1]

In [0]:
target_tensor

In [0]:
data_labels[0:20]

Convert to tensor

In [0]:
train_tokens_tensor, test_tokens_tensor, train_masks_tensor, test_masks_tensor = train_test_split(input_ids,attention_mask,test_size=TEST_SIZE,random_state=RANDOM_STATE)
train_y_tensor, test_y_tensor  = train_test_split(target_tensor,test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [0]:
input_ids

Dataloader 

In [0]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

Define model

In [0]:
class BertMultiClassifier(torch.nn.Module):
    def __init__(self, dropout=0.1, freeze_bert = False):
        super(BertMultiClassifier, self).__init__()
        # Need to define the right layer 
        self.bert = model_class.from_pretrained(pretrained_weights)
        #self.bert = BertModel.from_pretrained(pretrained_weights)
        #self.dropout = torch.nn.Dropout(dropout)

        if freeze_bert:
            for p in self.bert.parameters():
                p.requires_grad = False

        self.linear = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_DIM)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = last_hidden_states[0][:,0,:]
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        return linear_output

In [0]:
bert_clf = BertMultiClassifier(freeze_bert=False)
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss() # the same as log_softmax + NLLLoss.  Check BCEWithLogitsLoss()

In [45]:
losses = []
steps = []
step = 0
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        
        batch_loss = criterion(probas, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        #clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("{0}/{1} loss: {2} ".format(step_num, len(train_y_tensor) / BATCH_SIZE, train_loss / (step_num + 1)))
        losses.append(batch_loss.item())
        steps.append(step)
        step += 1
        
# Other option to save: torch.save(net.state_dict(), os.path.join(config["outputFolder"], config["outputFileName"]))


Epoch:  3
99/100.0 loss: 0.11000282391905784 


In [0]:
#convert_to_pickle(bert_clf, "/gdrive/My Drive/DataSet/ED6/DistillBERT91.pkl")

Evaluation

In [0]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)

        probas = bert_clf(token_ids, masks)
        
        bert_predicted += list(torch.max(probas,1)[1])



In [47]:
print(classification_report(test_y_tensor, bert_predicted))
print(confusion_matrix(test_y_tensor, bert_predicted))

              precision    recall  f1-score   support

           0       0.86      0.91      0.89       705
           1       0.93      0.89      0.91       895

    accuracy                           0.90      1600
   macro avg       0.89      0.90      0.90      1600
weighted avg       0.90      0.90      0.90      1600

[[641  64]
 [102 793]]


4. BERT freeze 

In [0]:
with torch.no_grad():
    last_hidden_states = BertModel(input_ids, attention_mask=attention_mask)
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, batch_1[category_col],test_size=TEST_SIZE,random_state=RANDOM_STATE)

In [72]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
lr_clf.score(test_features, test_labels)

0.736

5. 

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [157]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.238 (+/- 0.01)


To save time load BERT model and create logit for labeled and unlabeled data

In [0]:
### read data from your Google Drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# Read BERT model from pkl

class BertMultiClassifier(torch.nn.Module):
    def __init__(self, dropout=0.1):
        super(BertMultiClassifier, self).__init__()
        # Need to define the right layer 
        self.bert = model_class.from_pretrained(pretrained_weights)
        #self.dropout = torch.nn.Dropout(dropout)
        self.linear = torch.nn.Linear(HIDDEN_SIZE, OUTPUT_DIM)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, input_ids, attention_mask):
        last_hidden_states = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = last_hidden_states[0][:,0,:]
        #dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(pooled_output)
        proba = self.sigmoid(linear_output)
        return linear_output

bert_clf = load_from_pickle("/gdrive/My Drive/Colab Notebooks/STS2/DistillBERT.pkl")

6. Distill with training

In [0]:
# Create logits
train_logits = build_bert_logits(train_texts, bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [0]:
# Run regression 
distilled_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LinearRegression()).fit(train_texts, train_logits)
distilled_predicted_logits = distilled_model.predict(test_texts)
distilled_bert_predicted=torch.max(torch.tensor(distilled_predicted_logits),1)[1]
print(classification_report(test_y_tensor, distilled_bert_predicted))


              precision    recall  f1-score   support

           0       0.76      0.67      0.72       175
           1       0.77      0.84      0.80       225

    accuracy                           0.77       400
   macro avg       0.76      0.75      0.76       400
weighted avg       0.76      0.77      0.76       400



7. Distill with unlabeled

In [0]:
# Create logits
unlabel_logits = build_bert_logits(unlabel_texts, bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [1]:
# Run regression
from sklearn import linear_model
unlabel_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LinearRegression()).fit(unlabel_texts, unlabel_logits)
unlabel_predicted_logits = unlabel_model.predict(test_texts)
unlabel_bert_predicted=torch.max(torch.tensor(unlabel_predicted_logits),1)[1]
print(classification_report(test_y_tensor,unlabel_bert_predicted))

NameError: ignored

8. Distill with unlabel with Glove

In [57]:
unlabel_features_glove = AvgGlove(unlabel_texts) # output #sentences in Text x 100 features 
test_features_glove = AvgGlove(test_texts)      # output #sentences in Text x 100 features

Glove_distill_model=LinearRegression()
Glove_distill_model.fit(unlabel_features_glove, unlabel_logits)
Glove_distill_predicted_logits = Glove_distill_model.predict(test_features_glove)
Glove_distill_predicted = torch.max(torch.tensor(Glove_distill_predicted_logits),1)[1]
print(classification_report(test_y_tensor, Glove_distill_predicted))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77       503
           1       0.79      0.69      0.74       497

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.76      0.76      0.76      1000



9. Distill with data aug - 6 ED dataset

In [0]:
# Create logits
aug_logits = build_bert_logits(aug_texts, bert_clf, tokenizer, batch_size=BATCH_SIZE)

In [174]:
# Run regression
aug_model = make_pipeline(CountVectorizer(ngram_range=(1,2)), LinearRegression()).fit(aug_texts, aug_logits)
aug_predicted_logits = aug_model.predict(test_texts)
aug_bert_predicted=torch.max(torch.tensor(aug_predicted_logits),1)[1]
print(classification_report(test_y_tensor,aug_bert_predicted))

              precision    recall  f1-score   support

           0       0.49      0.07      0.13       229
           1       0.18      0.18      0.18       175
           2       0.42      0.76      0.54       517
           3       0.55      0.33      0.41       129
           4       0.58      0.43      0.50       485
           5       0.58      0.23      0.33        65

    accuracy                           0.44      1600
   macro avg       0.47      0.33      0.35      1600
weighted avg       0.47      0.44      0.41      1600



Init/Build train_y_tensor and test_y_tensor

Distill with unlabeled samples + training 

In [0]:
mix_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LinearRegression()).fit(np.concatenate((train_texts,unlabeled_texts),axis=0), np.concatenate((train_logits_load,unlabeled_logits.to_numpy())))
mix_predicted_logits = mix_model.predict(test_texts)
mix_bert_predicted=torch.max(torch.tensor(mix_predicted_logits),1)[1]
print(classification_report(test_y_tensor,mix_bert_predicted))

Distill with STS data

In [0]:
def build_bert_logits(texts, bert_clf,tokenizer, batch_size):

  tokenized = texts.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  tokenized_limted = []
  for i in tokenized.values:
    if len(i)>max_bert_len:
      tokenized_limted.append(i[:max_bert_len])
    else:
      tokenized_limted.append(i)

  padded = np.array([i + [0]*(max_bert_len-len(i)) for i in tokenized_limted])

  attention_mask = np.where(padded != 0, 1, 0)

  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  #tokens_tensor = torch.tensor(input_ids)
  #masks_tensor = torch.tensor(attention_mask)

  dataset = TensorDataset(input_ids, attention_mask)
  dataloader = DataLoader(dataset, batch_size)

  bert_clf.eval()
  list_logits = []
  with torch.no_grad():
      for step_num, batch_data in enumerate(dataloader):

          token_ids, masks = tuple(t for t in batch_data)

          logits = bert_clf(token_ids, masks)
        
          list_logits += list(logits)

          print("{0}/{1}".format(step_num, len(dataset) / batch_size))

  list_logits_numpy= (i.numpy() for i in list_logits)
  logits = np.vstack(list_logits_numpy)
  return(logits)


**N-gram plus Glove embedding** 
Example code: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb 

https://colab.research.google.com/github/MosheWasserb/PyTorchNotbooks/blob/master/Glove_CNN_MultiClass.ipynb#scrollTo=tnUazzVHSuB6